In [1]:
# imports
import sys
sys.path.append('../')

from glob import glob
from src.load_data import master_load
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Activation, BatchNormalization, Conv1D, Conv2D,
                                     Dense, Dropout, Flatten, LSTM, MaxPooling1D,
                                     MaxPooling2D, Reshape, TimeDistributed)
from tensorflow.math import confusion_matrix

In [2]:
# set plotting params
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 32
plt.rcParams['axes.labelsize'] = 32
plt.rcParams['axes.titlesize'] = 32
plt.rcParams['xtick.labelsize'] = 24
plt.rcParams['ytick.labelsize'] = 24
plt.rcParams['legend.fontsize'] = 32
plt.rcParams['figure.titlesize'] = 48

In [41]:
model_df = pd.DataFrame({'loss': [0], 'accuracy': [0], 'val_loss': [0], 'val_accuracy': [0], 'model_name': [0]})

In [51]:
def fit_model(model, X_train, y_train, X_test, y_test, epochs=5, batch_size=32, model_type='numpy', df=model_df):
    fitted = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))
    score = fitted.model.evaluate(X_test, y_test, batch_size=batch_size)

    model_name = f'../models/{model_type}/acc_{round(score[1], 3)}-loss_{round(score[0], 2)}-{int(time.time())}.model'

    temp_df = pd.DataFrame.from_dict(fitted.history)
    temp_df['model_name'] = model_name
    pd.concat([df, temp_df])
    print(df)

    fitted.model.save(model_name)
    print('done')

In [6]:
# load image data
train_l_paths = glob('../img/data/train/left/*')
train_n_paths = glob('../img/data/train/none/*')
train_r_paths = glob('../img/data/train/right/*')
test_l_paths = glob('../img/data/test/left/*')
test_n_paths = glob('../img/data/test/none/*')
test_r_paths = glob('../img/data/test/right/*')

In [7]:
def load_image(path):
    rgba_image = Image.open(path)
    rgb_image = rgba_image.convert('RGB')
    return np.asarray(rgb_image)

In [9]:
def stack_images(paths_array):
    array = []
    
    for i in paths_array:
        array.append(load_image(i))
    
    return np.array(array)

In [10]:
def combine(left, none, right):
    combined = []
    for i in left:
        combined.append([i, np.array([1, 0, 0])])

    for i in none:
        combined.append([i, np.array([0, 1, 0])])

    for i in right:
        combined.append([i, np.array([0, 0, 1])])
        
    return np.array(combined, dtype=object)

In [46]:
def load_img_data(left_train, none_train, right_train, left_test, none_test, right_test):
    left_T_images = stack_images(left_train)
    none_T_images = stack_images(none_train)
    right_T_images = stack_images(right_train)

    left_t_images = stack_images(left_test)
    none_t_images = stack_images(none_test)
    right_t_images = stack_images(right_test)
    
    train = combine(left_T_images, none_T_images, right_T_images)
    test = combine(left_t_images, none_t_images, right_t_images)
    
    np.random.shuffle(train)
    np.random.shuffle(test)
    
    X_train = []
    y_train = []
    
    X_test = []
    y_test = []

    for feature, target in train:
        X_train.append(feature)
        y_train.append(target)
    
    for feature, target in test:
        X_test.append(feature)
        y_test.append(target)
        
    X_train = np.array(X_train) / 255
    y_train = np.array(y_train)
    X_test = np.array(X_test) / 255
    y_test = np.array(y_test)

    return X_train, y_train, X_test, y_test

In [47]:
# load image data
X_train_img, y_train_img, X_test_img, y_test_img = load_img_data(train_l_paths, train_n_paths, train_r_paths,
                                                                 test_l_paths, test_n_paths, test_r_paths)

In [49]:
# load fft data
X_train, X_test, y_train, y_test = master_load()

In [4]:
seven_XT = X_train[:, 6, :]
fifteen_XT = X_train[:, 14, :]

seven_Xt = X_test[:, 6, :]
fifteen_Xt = X_test[:, 14, :]

In [5]:
X_train_slice = np.stack((seven_XT, fifteen_XT), axis=1)
X_test_slice = np.stack((seven_Xt, fifteen_Xt), axis=1)

In [35]:
model = Sequential()

model.add(LSTM(32, return_sequences=True, input_shape=X_train.shape[1:]))  # numpy
model.add(Dropout(0.2))

model.add(LSTM(8, return_sequences=True))
model.add(Dropout(0.2))

model.add(TimeDistributed(
    Dense(32, activation='relu')))

# model.add(TimeDistributed(
#     Dense(8, activation='relu')))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [36]:
fit_model(model, X_train, y_train, X_test, y_test, epochs=2)

Epoch 1/2
8895/8895 [==============================] - 79s 9ms/step - loss: 0.9861 - accuracy: 0.4821 - val_loss: 1.0801 - val_accuracy: 0.3872
Epoch 2/2
1102/1102 [==============================] - 2s 2ms/step - loss: 1.1544 - accuracy: 0.3834


INFO:tensorflow:Assets written to: ../models/numpy/acc_0.383-loss_1.15-1628280050.model/assets


INFO:tensorflow:Assets written to: ../models/numpy/acc_0.383-loss_1.15-1628280050.model/assets


done


In [55]:
# modified
model = Sequential()

model.add(Conv2D(64, (3, 3), input_shape=X_train_img.shape[1:]))
model.add(Activation('selu'))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('selu'))
model.add(Dropout(0.2))

model.add(MaxPooling2D((3, 3), padding='same'))
model.add(Flatten())

model.add(Dense(64))
model.add(Activation('selu'))

model.add(Dense(16))
model.add(Activation('selu'))
model.add(Dropout(0.5))

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [56]:
fit_model(model, X_train_img, y_train_img, X_test_img, y_test_img, epochs=2, model_type='images')

Epoch 1/2
35/35 [==============================] - 511s 14s/step - loss: 23.0176 - accuracy: 0.3453 - val_loss: 95.2675 - val_accuracy: 0.3333
Epoch 2/2
5/5 [==============================] - 16s 3s/step - loss: 119.7239 - accuracy: 0.3333
   loss  accuracy  val_loss  val_accuracy  model_name
0     0         0         0             0           0
INFO:tensorflow:Assets written to: ../models/images/acc_0.333-loss_119.72-1628283532.model/assets


INFO:tensorflow:Assets written to: ../models/images/acc_0.333-loss_119.72-1628283532.model/assets


done


In [10]:
# modified
model = Sequential()

model.add(Conv1D(64, (2), input_shape=X_train.shape[1:]))
model.add(Activation('relu'))

model.add(Conv1D(64, (1)))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=(2), padding='same'))

model.add(Conv1D(64, (1)))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=(2), padding='same'))

model.add(Flatten())

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(16))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [11]:
epochs = 50
batch_size = 32

fitted = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))
score = fitted.model.evaluate(X_test, y_test, batch_size=batch_size)

model_name = f'../models/numpy/acc_{round(score[1], 3)}-loss_{round(score[0], 2)}-{int(time.time())}.model'

temp_df = pd.DataFrame.from_dict(fitted.history)
temp_df['model_name'] = model_name
df = pd.concat([df, temp_df])
fitted.model.save(model_name)

print('done')

Epoch 1/50
8895/8895 [==============================] - 57s 6ms/step - loss: 1.0991 - accuracy: 0.3339 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 2/50
8895/8895 [==============================] - 64s 7ms/step - loss: 1.0986 - accuracy: 0.3341 - val_loss: 1.0988 - val_accuracy: 0.3333
Epoch 3/50
8895/8895 [==============================] - 70s 8ms/step - loss: 1.0987 - accuracy: 0.3341 - val_loss: 1.0987 - val_accuracy: 0.3333
Epoch 4/50
8895/8895 [==============================] - 69s 8ms/step - loss: 1.0987 - accuracy: 0.3332 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 5/50
8895/8895 [==============================] - 68s 8ms/step - loss: 1.0987 - accuracy: 0.3343 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 6/50
8895/8895 [==============================] - 71s 8ms/step - loss: 1.0986 - accuracy: 0.3330 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 7/50
8895/8895 [==============================] - 73s 8ms/step - loss: 1.0987 - accuracy: 0.3337 - val_loss: 1.0986 - val_ac

INFO:tensorflow:Assets written to: ../models/numpy/acc_0.333-loss_1.1-1628240895.model/assets


done


In [12]:
# stolen 1
model = Sequential()

model.add(Conv1D(128, (1), input_shape=X_train.shape[1:]))
model.add(Activation('relu'))

model.add(Conv1D(64, (2)))
model.add(Activation('relu'))

model.add(MaxPooling1D(pool_size=(2), padding='same'))

model.add(Conv1D(64, (1)))
model.add(Activation('relu'))

model.add(MaxPooling1D(pool_size=(2), padding='same'))

model.add(Flatten())

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(16))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [13]:
epochs = 50
batch_size = 32

fitted = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))
score = fitted.model.evaluate(X_test, y_test, batch_size=batch_size)

model_name = f'../models/numpy/acc_{round(score[1], 3)}-loss_{round(score[0], 2)}-{int(time.time())}.model'

temp_df = pd.DataFrame.from_dict(fitted.history)
temp_df['model_name'] = model_name
df = pd.concat([df, temp_df])
fitted.model.save(model_name)

print('done')

Epoch 1/50
8895/8895 [==============================] - 74s 8ms/step - loss: 1.0989 - accuracy: 0.3339 - val_loss: 1.0987 - val_accuracy: 0.3333
Epoch 2/50
8895/8895 [==============================] - 71s 8ms/step - loss: 1.0986 - accuracy: 0.3344 - val_loss: 1.0989 - val_accuracy: 0.3333
Epoch 3/50
8895/8895 [==============================] - 68s 8ms/step - loss: 1.0987 - accuracy: 0.3336 - val_loss: 1.0987 - val_accuracy: 0.3333
Epoch 4/50
8895/8895 [==============================] - 66s 7ms/step - loss: 1.0987 - accuracy: 0.3343 - val_loss: 1.0987 - val_accuracy: 0.3333
Epoch 5/50
8895/8895 [==============================] - 69s 8ms/step - loss: 1.0986 - accuracy: 0.3356 - val_loss: 1.0988 - val_accuracy: 0.3333
Epoch 6/50
8895/8895 [==============================] - 69s 8ms/step - loss: 1.0987 - accuracy: 0.3325 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 7/50
8895/8895 [==============================] - 68s 8ms/step - loss: 1.0987 - accuracy: 0.3350 - val_loss: 1.0988 - val_ac

INFO:tensorflow:Assets written to: ../models/numpy/acc_0.333-loss_1.1-1628244246.model/assets


done


In [19]:
reshape = (-1, 2, 60)

train_X = np.clip(np.array(X_train).reshape(reshape), -10, 10) / 10
test_X = np.clip(np.array(X_test).reshape(reshape), -10, 10) / 10

In [23]:
# stolen 2
model = Sequential()

model.add(Conv1D(128, (1), padding='same', input_shape=train_X.shape[1:]))
model.add(Activation('relu'))

model.add(Conv1D(64, (5), padding='same'))
model.add(Activation('relu'))

model.add(Conv1D(16, (5), padding='same'))
model.add(Activation('relu'))

model.add(Conv1D(8, (5), padding='same'))
model.add(Activation('relu'))

model.add(Conv1D(3, (2)))
model.add(Reshape((3,)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [24]:
epochs = 50
batch_size = 32

fitted = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))
score = fitted.model.evaluate(X_test, y_test, batch_size=batch_size)

model_name = f'../models/numpy/acc_{round(score[1], 3)}-loss_{round(score[0], 2)}-{int(time.time())}.model'

temp_df = pd.DataFrame.from_dict(fitted.history)
temp_df['model_name'] = model_name
df = pd.concat([df, temp_df])
fitted.model.save(model_name)

print('done')

Epoch 1/50


ValueError: in user code:

    /Users/josiah/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /Users/josiah/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/josiah/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/josiah/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/josiah/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/josiah/opt/anaconda3/lib/python3

In [ ]:
def residual_block(x, filters, conv_num=3, activation="relu"):
    # Shortcut
    s = keras.layers.Conv1D(filters, 1, padding="same")(x)
    for i in range(conv_num - 1):
        x = keras.layers.Conv1D(filters, 3, padding="same")(x)
        x = keras.layers.Activation(activation)(x)
    x = keras.layers.Conv1D(filters, 3, padding="same")(x)
    x = keras.layers.Add()([x, s])
    x = keras.layers.Activation(activation)(x)
    return keras.layers.MaxPool1D(pool_size=2, strides=2, padding='same')(x)

In [ ]:
def build_model(input_shape, num_classes):
    inputs = keras.layers.Input(shape=input_shape, name="input")

    x = residual_block(inputs, 16, 2)
    x = residual_block(x, 32, 2)
    x = residual_block(x, 64, 3)
    x = residual_block(x, 128, 3)
    x = residual_block(x, 128, 3)

    x = keras.layers.MaxPooling1D(pool_size=(2), padding='same')(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.Dense(128, activation="relu")(x)

    outputs = keras.layers.Dense(num_classes, activation="softmax", name="output")(x)

    return keras.models.Model(inputs=inputs, outputs=outputs)

In [ ]:
model = build_model((X_train.shape[1:]), 3)

In [ ]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
epochs = 50
batch_size = 32

fitted = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))
score = fitted.model.evaluate(X_test, y_test, batch_size=batch_size)

model_name = f'../models/numpy/acc_{round(score[1], 3)}-loss_{round(score[0], 2)}-{int(time.time())}.model'

temp_df = pd.DataFrame.from_dict(fitted.history)
temp_df['model_name'] = model_name
df = pd.concat([df, temp_df])
fitted.model.save(model_name)

print('done')

In [ ]:
best_model = sorted(glob('../models/numpy/*.model'))[-1]

In [ ]:
model = tf.keras.models.load_model(best_model)

y_pred = np.argmax(model.predict(X_test), axis=-1)
y_true = np.argmax(y_test, axis=-1)
conf_mat = np.round(np.array(confusion_matrix(y_true, y_pred)) / 11750, 3)

In [ ]:
def plot_conf_mat(conf_mat):
    actions = ['left', 'none', 'right']

    fig, ax = plt.subplots(figsize=(16, 9))

    ax.matshow(conf_mat, cmap=plt.cm.RdYlGn)
    ax.set_xticklabels([""]+actions)
    ax.set_yticklabels([""]+actions)

    for i, val0 in enumerate(conf_mat.T):
        for ii, val1 in enumerate(val0):
            ax.text(i, ii, f"{val1}", va='center', ha='center')

    plt.title("Action Thought")
    plt.ylabel("Predicted Action")
    plt.show()

In [ ]:
plot_conf_mat(conf_mat)

# trained KNN, log_reg, GBC
# all trash, acc < 0.35